In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**Centro Universitário IESB

Pós Graduação em Ciência de Dados

Data Mining e Machine Learning II

Professor Marcos Vinicius Guimarães

Fernanda Fernandes Ministério**

Objetivo deste trabalho
Identificar as pessoas que possam ter problemas de inadimplência junto aos bancos na modalidade de habitação (Home Equity).

Definir um modelo que faça predição dos maus pagadores por meio de uma base de dados do Home Equity com aproximadamente 6.000 empréstimos anteriormente concedidos. Podemos usar estas informações identificando clientes que tenham deixado de pagar alguma parcela de empréstimo no passado para treinar modelos de aprendizado de máquina para prever probabilidades de pessoas que poderiam deixar de pagar a modalidade habitação (Home Equity) no futuro baseado em situações anteriores.

Assim vou propor um problema de classificação binária onde o modelo deverá prever se uma pessoa seria um pagador ruim.


Metodologia
A base de dados "Home Equity" possui  pessoas e dados de empréstimo de 5.960 empréstimos recentes. Para cada empréstimo existem 12 variáveis. A variável alvo (BAD) indica quando o cliente não pagou o empréstimo (1), e quando ele pagou (0).

Serão utilizados os modelos Random Forest Classifier, XGBosst e XGBoost com auxílio do GridSearchCV para otimização do modelo

Dicionário de Dados (em ordem alfabética)

BAD: 1 = client defaulted on loan 0 = loan repaid

CLAGE: Age of oldest trade line in months

CLNO: Number of credit lines

DEBTINC: Debt-to-income ratio

DELINQ: Number of delinquent credit lines

DEROG: Number of major derogatory reports

JOB: Six occupational categories

LOAN: Amount of the loan request

MORTDUE: Amount due on existing mortgage

NINQ: Number of recent credit lines

REASON: DebtCon = debt consolidation ; HomeImp = home improvement

VALUE: Value of current property

YOJ: Years at present job

In [ ]:
# importando o dataset
df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')
df.head()

In [ ]:
# Verificando as informações do dataset
df.shape, df.info()

Observa-se que a maior parte das colunas são valores numéricos

In [ ]:
# realizando análise exploratória

df.describe(include='all')

In [ ]:
# Verificando o percentual de empréstimos não pagos (BAD = 1)

print(df['BAD'].value_counts())
print(df['BAD'].value_counts(normalize=True))

In [ ]:
# Verificando estatísticas básicas dos valores de empréstimos. A média está em 18.607.

df['LOAN'].describe()

In [ ]:
# Fazer tratamento dos Dados verificando a existência de missing values ou valores null

MissingValues =df.isnull().sum().rename_axis('Colunas').reset_index(name='Missing Values')
MissingValues

In [ ]:
# utilizando a biblioteca Pandas Profiling para gerar um report com análise de todas os campos e suas principais estatísticas.

import pandas_profiling as pp

pp.ProfileReport(df)

In [ ]:
# avaliação das variáveis numéricas por meio de histogramas
import matplotlib.pyplot as plt
%matplotlib inline
df.hist(figsize=(20,10))

Observa-se que a base está desbalanceada e que possuímos uma quantidade pequena de maus pagadores (1)

In [ ]:
# Criando um dataframe para empréstimos que não foram pagos

df_bad = df[df['BAD'] == 1]

In [ ]:
# Verificando o Total dos empréstimos não pagos (20.120.400). Este total é relevante para estudar os potenciais maus pagadores.

df_bad['LOAN'].sum()

In [ ]:
# Explorando as informações
df['DELINQ'].value_counts()

In [ ]:
print(df['LOAN'].sum())
print(df[df['DELINQ'] != 0.0]['LOAN'].sum())
print(df[df['DELINQ'] == 0.0]['LOAN'].sum())

In [ ]:
# Distribuição por profissão
%matplotlib inline

df['JOB'].value_counts().plot.bar()

In [ ]:
# Distribuição por motivo do empréstimo
%matplotlib inline

df['REASON'].value_counts().plot.bar()

Observa-se que no gráfico de correlação abaixo as variáveis não apresentam correlação fortes entre si. A melhor correlação apresentada foi entre as variáveis MORTDUE e VALUE (valor da hipotéca devido e o valor da propriedade).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotando a correlação

# Aumentando a area do grafico
f, ax = plt.subplots(figsize=(15,6))
sns.heatmap(df.corr(), annot=True, fmt='.2f', linecolor='red', ax=ax, lw=1)

In [ ]:
# Identificando valores de hipoteta devido nulo.

df[df['MORTDUE'].isna()]

In [ ]:
# Substituindo Nan por 0

df.fillna(0, inplace = True)

In [ ]:
# criando uma nova base para proteger a base original

df_n = df

In [ ]:
# Criando uma nova coluna onde 0 não é maior e 1 é maior que, para o campo da hipotéca ser maior que o valor da propriedade

df_n['HIP_M_PROP'] = df_n['VALUE'] - df_n['MORTDUE']

HIP_M_PROP = []

In [ ]:
# determinar as categorias
for valor in df_n['HIP_M_PROP']:
    if valor <  0.0:
        HIP_M_PROP.append(1)
    elif valor >= 0.0:
        HIP_M_PROP.append(0)
        
df_n['HIP_M_PROP'] = HIP_M_PROP

In [ ]:
# Criando nova coluna com valores 1 (sim) e 0 (não) para definir se o valor do empréstimo é maior que o valor da hipoteca

df_n['LOAN_M_HIP'] = df_n['LOAN'] - df_n['MORTDUE']

LOAN_M_HIP = []

# determinar as categorias
for valor in df_n['LOAN_M_HIP']:
    if valor <  0.0:
        LOAN_M_HIP.append(1)
    elif valor >= 0.0:
        LOAN_M_HIP.append(0)
        
df_n['LOAN_M_HIP'] = LOAN_M_HIP

Observamos agora que as correlações de variáveis com a variável resposta BAD não são significativas
Temos então dois cenários possíveis para predição:
> pegar todas as variáveis, exceto  'BAD','REASON', 'JOB' e preparar um treinamento para encontrar um padrão que melhor gere um modelo capaz de prever a saída para novos dados. Será adotado X para os valores das entradas (features) e y para os valores das saídas.
> outra possibilidade é pegar DEBTINC, e analisar a apresentada correlação média com a variável resposta BAD.

In [ ]:
features = df_n.columns.difference(['BAD','REASON','JOB'])

X = df_n[features].values
y = df_n['BAD'].values

In [ ]:
# dividir a base em treino e teste

import pandas as pd

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_n.drop('BAD',
                                                    axis=1),
                                                    df_n['BAD'],
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
# criação das bases e seus tamanhos
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
# base de treino
x_train.head()

In [ ]:
# Importando o método do scikitlearn para divisão do dataframe de treino em treino e validação

from sklearn.model_selection import train_test_split

In [ ]:
# Dividindo a base de treino para teste

train, valid = train_test_split(x_train, random_state=42)

In [ ]:
# base de validação
valid.head()

In [ ]:
valid.shape

In [ ]:
train.shape

In [ ]:
# árvore de decisão
from sklearn.tree import DecisionTreeClassifier

classifier_dt = DecisionTreeClassifier(random_state=1986,
                           criterion='gini',
                           max_depth=3)
classifier_dt.fit(X, y)

In [ ]:
# validar modelo
from sklearn.model_selection import cross_val_score

scores_dt = cross_val_score(classifier_dt, X, y,
                            scoring='accuracy', cv=6)

print(scores_dt.mean())

Observa-se neste caso que a acurácia é ruim

In [ ]:
# predição com Ensemble

from sklearn.ensemble import RandomForestClassifier

classifier_rf = RandomForestClassifier(random_state=1986,
                           criterion='gini',
                           max_depth=10,
                           n_estimators=30,
                           n_jobs=-1)
scores_rf = cross_val_score(classifier_rf, X, y,
                            scoring='accuracy', cv=6)

print(scores_rf.mean())

Ficou parecido o resultado com a aplicação do método CrossValidation, o que significa que o modelo continua sem boa acurácia.

In [ ]:
# tentando uma nova modelagem e medindo a importância da features

classifier_rf.fit(X, y) 

features_importance = zip(classifier_rf.feature_importances_, features)
for importance, feature in sorted(features_importance, reverse=True):
    print("%s: %f%%" % (feature, importance*100))

Como o resultado sobre DEBTINC ficou melhor, vamos focar nesta variável como explicativa

In [ ]:
# nova modelagem com Variável resposta: BAD e Variável explicativa: DEBTINC

In [ ]:
# Identificando a frequência da variável DEBTINC

df['DEBTINC'].value_counts()

In [ ]:
# Separando os dataframes onde o count é nulo

teste = df[df['DEBTINC'] == 0.0]

treino = df[df['DEBTINC'] != 0.0]

In [ ]:
treino.shape

In [ ]:
teste.shape

In [ ]:
# Modelo RandomForest

# método do scikitlearn para divisão e instanciando o modelo

from sklearn.model_selection import train_test_split

rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, oob_score=True, random_state=42)

In [ ]:
# Removendo as colunas de resposta

removed_cols = ['BAD','DEBTINC','JOB','REASON']

In [ ]:
# Criar a lista da colunas de entrada

feats = [c for c in train.columns if c not in removed_cols]

In [ ]:
# Treinamento do modelo com as variáveis de entrada e as de resposta
rf.fit(treino[feats], treino['BAD'])

In [ ]:
# Previsão da variável de teste usando o modelo treinado
teste['BAD'] = rf.predict(teste[feats]).astype(int)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier_rf = RandomForestClassifier(random_state=1986,
                           criterion='gini',
                           max_depth=10,
                           n_estimators=30,
                           n_jobs=-1)
scores_rf = cross_val_score(classifier_rf, X, y,
                            scoring='accuracy', cv=6)

print(scores_rf.mean())

In [ ]:
# Este modelo apresentou um desempenho melhor no percentual de predição.
# Há uma margem de diferença entre a base primária e a base testada de 10%

In [ ]:
# Verificação das previsões
teste['BAD'].value_counts(normalize=True)

In [ ]:
df['BAD'].value_counts(normalize=True)